In [40]:
import pandas as pd
from sqlalchemy import create_engine, text
from datetime import datetime
import hashlib
from dotenv import load_dotenv
import os

# Load DB credentials
load_dotenv()

PG_USER = os.getenv("PG_USER")
PG_PASSWORD = os.getenv("PG_PASSWORD")
PG_HOST = os.getenv("PG_HOST")
PG_PORT = os.getenv("PG_PORT", "5432")
PG_DB = os.getenv("PG_DB")


# --- Setup connection ---
conn_str = f"postgresql://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DB}"
engine = create_engine(conn_str)  # replace conn_str with your actual connection string

# --- Step 1: Create tables ---

with open("../sql/create_cp_cpk_tables.sql", "r") as f:
    create_tables_sql = f.read()

with engine.begin() as conn: 
    conn.execute(text(create_tables_sql))


In [41]:
# --- Step 2: Query latest 30 measurements per material-variable ---

qc_query = """
WITH ranked_data AS (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY material_no, variable ORDER BY measurement_date DESC) AS rank
    FROM manufacturing_qc
    WHERE lsl IS NOT NULL AND usl IS NOT NULL
)
SELECT * FROM ranked_data WHERE rank <= 10
ORDER BY material_no, variable, measurement_date;
"""

qc_df = pd.read_sql(qc_query, engine)


In [42]:
# --- Step 3: Calculate moving range, sigma_within, mean_value, Cp, Cpk ---

qc_df['moving_range'] = qc_df.groupby(['material_no', 'variable'])['measurement_value'].diff().abs()

qc_df['sigma_within'] = qc_df.groupby(['material_no', 'variable'])['moving_range'].transform('mean') / 1.128
qc_df['mean_value'] = qc_df.groupby(['material_no', 'variable'])['measurement_value'].transform('mean')

qc_df['cp'] = (qc_df['usl'] - qc_df['lsl']) / (6 * qc_df['sigma_within'])

def calc_cpk(row):
    if row['sigma_within'] == 0 or pd.isna(row['sigma_within']):
        return None
    return min(
        (row['usl'] - row['mean_value']) / (3 * row['sigma_within']),
        (row['mean_value'] - row['lsl']) / (3 * row['sigma_within'])
    )

qc_df['cpk'] = qc_df.apply(calc_cpk, axis=1)

calc_time = datetime.now()
qc_df['calculation_time'] = calc_time
qc_df['calculation_id'] = qc_df.apply(
    lambda r: hashlib.md5(f"{r['material_no']}_{r['variable']}_{calc_time.isoformat()}".encode()).hexdigest(),
    axis=1
)



In [43]:
# --- Step 4: Write wide table ---
wide_cols = [
    'calculation_id', 'work_order', 'material_no', 'material_type', 'variable',
    'measurement_date', 'measurement_value', 'lsl', 'usl', 'moving_range',
    'sigma_within', 'mean_value', 'cp', 'cpk', 'calculation_time'
]

qc_df[wide_cols].to_sql('cp_cpk_wide', engine, if_exists='append', index=False)
print("✅ Inserted into cp_cpk_wide.")


✅ Inserted into cp_cpk_wide.


In [44]:
# --- Step 5: Aggregate summary table ---

summary_df = (
    qc_df.groupby(['material_no', 'material_type', 'variable'])
    .agg({
        'cp': 'mean',
        'cpk': 'mean'
    }).reset_index()
)

summary_df['calculation_time'] = calc_time
# Pick the first calculation_id from each group — since it's the same for all rows in the group
summary_ids = (
    qc_df.groupby(['material_no', 'material_type', 'variable'])['calculation_id']
    .first()
    .reset_index()
)

# Merge that into the summary_df
summary_df = summary_df.merge(summary_ids, on=['material_no', 'material_type', 'variable'])

# Add calculation_time if not already there
summary_df['calculation_time'] = calc_time



In [45]:

# --- Step 6: Write summary table ---
summary_cols = [
    'calculation_id', 'material_no', 'material_type', 'variable', 'cp', 'cpk',
    'calculation_time'
]

summary_df[summary_cols].to_sql('cp_cpk_summary', engine, if_exists='append', index=False)
print("✅ Inserted into cp_cpk_summary.")

✅ Inserted into cp_cpk_summary.
